# G-Research Crypto forecasting competition

### コンペ概要

目的：暗号通貨によるリターンの予測<br>
暗号通貨に関する時系列の価格履歴を使用して、価格が上がるか下がるか、どれだけの資産がリターンするかを予測します。<br>
本コンペでは、14種の暗号通貨についてのデータが提供されています。<br>

データの特性
- 時系列データ。
- 暗号通貨の変動は、他の暗号通貨や暗号通貨市場全体に対して影響している。
- 株価と同様に、通貨の価格はランダムウォークパターンに従っているようにも見える。

<br>

チュートリアルを参考に、submission提出までをシンプルなモデルを作成します。<br>

**参考**
- [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)
- [Data Description](https://www.kaggle.com/c/g-research-crypto-forecasting/data)
- [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

日本語EDA：[【日本語】G-research_Data_Outline](https://www.kaggle.com/kitopl/g-research-data-outline)<br>

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

### データ概要確認

In [ ]:
data = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
data.head()

- **timestamp**：1970-01-01 00:00:00.000 UTCからの経過秒数を示す。データは分スケールのため、60の倍数になっているようです。
- **Asset_ID**：暗号通貨を識別するID(例：ビットコインの場合はAsset_ID = 1)。 Asset_IDと暗号通貨の対応関係はasset_details.csvで参照できます。
- **Count**：時間間隔（直前）の取引の総数。
- **Open**：指定時間間隔における始値(USD)。
- **High**：指定時間間隔における最高価格(USD)。
- **Low**：指定時間間隔における最低価格(USD)。
- **Close**：指定時間間隔における終値(USD)。
- **Volume**：購入または販売された通貨の数量。
- **VWAP**：指定時間間隔における通貨の重みづけ平均価格。
- **Target**：15分間の対数収益率から算出。
timestamp、Asset_IDの複合キーが、データを識別するインデックになります。

In [ ]:
data.describe()

Asset_IDと暗号通貨の対応<br>
Weightは、目的変数となっているLogReturnを算出する際に使用します。

In [ ]:
asset_tmp = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset = asset_tmp.set_index('Asset_ID').sort_index()
asset

In [ ]:
# 各通貨毎に欠損値、欠損データを確認
missing_list = []
for asset_id, asset_name in zip(asset.index, asset.loc[:, 'Asset_Name']):
    asset_df = data[data["Asset_ID"]==asset_id].set_index("timestamp")
    gap_df = (asset_df.index[1:]-asset_df.index[:-1]).value_counts().reset_index()
    drop_row = int(((gap_df['index']/60-1)*gap_df['timestamp']).sum())
    missing_list.append([asset_name, asset_df.shape[0]] + asset_df.isna().sum().tolist() + [drop_row])

missing_df = pd.DataFrame(missing_list)
missing_df.columns = ["Asset_Name", "TotalRows", 'Missing_Asset_ID', 'Missing_Count', 'Missing_Open', 
                      'Missing_High', 'Missing_Low', 'Missing_Close', 'Missing_Volume', 'Missing_VWAP', 'Missing_Target', 'Missing_Row']
missing_df

### 学習
チュートリアルで紹介されているupper_shadowとlower_shadowと特徴量に加えてみます。<br>
通貨毎で特徴が異なるので、学習も通貨毎に分けて行います。

In [ ]:
# チュートリアルより
# 上ひげ、下ひげを計算
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def data_split_by_asset(df_train, asset_id):
    df_asset = data[data["Asset_ID"] == asset_id]
    
    df_asset['Upper_Shadow'] = upper_shadow(df_asset)
    df_asset['Lower_Shadow'] = lower_shadow(df_asset)
    df_asset = df_asset.dropna(how='any')
    
    X = df_asset.drop(['timestamp', 'Asset_ID', 'Target'], axis=1)
    y = df_asset['Target']

    model = LGBMRegressor(n_estimators=10)
    model.fit(X, y)
    return X, y, model

In [ ]:
X_dict = {}
y_dict = {}
model_dict = {}

for asset_id, asset_name in zip(asset.index, asset['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = data_split_by_asset(data, asset_id)    
    X_dict[asset_id] = X
    y_dict[asset_id] = y
    model_dict[asset_id] = model

### テストデータの特徴量取得、予測
Cryptoコンペでは、データと一緒に提供されているgresearch_crypto Python moduleを使用する必要があります。<br>
これは、将来のデータを使用して予測を行わないようにするためであるようです。<br>
[API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)に記載の通り、predictionを行います。<br>

In [ ]:
import gresearch_crypto

env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [ ]:
df_test_entire = []

for i, (df_test, sample_prediction_df) in enumerate(iter_test):
    # Asset_IDでモデルを分割しているため、1データ毎で予測
    for j, test_row in df_test.iterrows():

        model = model_dict[test_row['Asset_ID']]
        test_row['Upper_Shadow'] = upper_shadow(test_row)
        test_row['Lower_Shadow'] = lower_shadow(test_row)

        X_test = test_row.drop(['timestamp', 'Asset_ID', 'row_id']).values.reshape(1, -1)
        y_pred = model.predict(X_test)

        sample_prediction_df.loc[sample_prediction_df['row_id'] == test_row['row_id'], 'Target'] = y_pred

    df_test_entire.append(df_test)

    # submissions提出
    env.predict(sample_prediction_df)